In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
tf.__version__

'2.3.1'

In [14]:
batch_size = 64
split = 0.2
epochs = 5

In [3]:
class celeb_dataframe():
    '''Wraps the celebA dataset, allowing an easy way to:
       - Select the features of interest,
       - Split the dataset into 'training', 'test' or 'validation' partition.
    '''
    def __init__(self,main_folder = "/kaggle/input/celeba-dataset" , selected_features = [] , drop_features = []):
        self.main_folder = main_folder
        self.images_folder   = os.path.join(main_folder, 'img_align_celeba/img_align_celeba')
        self.attr_path = os.path.join(main_folder , "list_attr_celeba.csv")
        self.features_name = []
        self.prepare(drop_features , selected_features)
        
    def prepare(self,drop_features, selected_features ):
        
        #attributes selection
        if len(selected_features) == 0  :
            self.attributes = pd.read_csv(self.attr_path)
            self.num_features = 40
        else:
            self.num_features = len(selected_features)
            selected_features = selected_features.copy()
            selected_features.append("image_id")
            self.attributes = pd.read_csv(self.attr_path)[selected_features]
        
        #removing features
        if len(drop_features) != 0:
            for feature in drop_features:
                if feature in self.attributes:
                    self.attributes = self.attributes.drop(feature , axis = 1)
                    self.num_features -= 1
            
        self.attributes.set_index("image_id" , inplace = True)
        self.attributes.replace(to_replace = -1 , value = 0 , inplace = True)
        self.attributes["image_id"] = list(self.attributes.index)
        self.features_name = list(self.attributes.columns)[:-1]
        
        return self.attributes 
        
        
                
                
            
        
        
    

In [4]:
features  = ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
       'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
       'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
       'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young']

In [5]:
celeb_df = celeb_dataframe()
celeb = celeb_df.prepare(drop_features=[],selected_features= features) #taking all the features for training.
celeb.head(10)


,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,image_id
image_id,,,,,,,,,,,,,,,,,,,,,
000001.jpg,0,1,1,0,0,0,0,0,0,0,...,1,1,0,1,0,1,0,0,1,000001.jpg
000002.jpg,0,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,000002.jpg
000003.jpg,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,000003.jpg
000004.jpg,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,1,000004.jpg
000005.jpg,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,000005.jpg
000006.jpg,0,1,1,0,0,0,1,0,0,0,...,0,0,1,1,0,1,0,0,1,000006.jpg
000007.jpg,1,0,1,1,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,1,000007.jpg
000008.jpg,1,1,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,000008.jpg
000009.jpg,0,1,1,0,0,1,1,0,0,0,...,1,0,0,1,0,1,0,0,1,000009.jpg


In [7]:
print("Number of features:",len(celeb_df.features_name))

Number of features: 40


In [8]:
split_ind = int((1 - split) * celeb.shape[0])
train_df = celeb[:split_ind]
validation_df = celeb[split_ind:]

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# augumentations for training set:
train_datagen = ImageDataGenerator(rotation_range=20, 
                                   rescale=1./255, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale= 1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=celeb_df.images_folder,
    x_col='image_id',
    y_col=celeb_df.features_name,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='raw',
    shuffle = True
)

validation_generator = valid_datagen.flow_from_dataframe(dataframe=validation_df,
                                                         directory=celeb_df.images_folder,
                                                         x_col='image_id',
                                                         y_col=celeb_df.features_name,
                                                         target_size=(224, 224),
                                                         batch_size=batch_size,
                                                         class_mode='raw'
                                                        )

                                                         


Found 162079 validated image filenames.
Found 40520 validated image filenames.


In [10]:
from tensorflow.keras.applications.mobilenet_v2  import MobileNetV2
from tensorflow.keras.layers import Dense , BatchNormalization , Dropout

def classifier(num_features):
    base = MobileNetV2(input_shape = (224,224,3),
                      weights = None,
                      include_top=False,
                      pooling = "avg")
    
    x = base.output
    x = Dense(1536, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    top = Dense(num_features, activation='sigmoid')(x)
    classifier = tf.keras.models.Model(base.input,top)
    
    return classifier

In [11]:
num_features = len(celeb_df.features_name)
cls = classifier(num_features)
cls.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [12]:
cls.compile(loss='binary_crossentropy',
              optimizer= tf.keras.optimizers.Adam(0.001),
              metrics='binary_accuracy')

In [13]:
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint

earlystop = EarlyStopping(monitor="val_binary_accuracy", patience= 3)
os.mkdir("/kaggle/working/ckpts/")
checkpoint_filepath = "/kaggle/working/ckpts/" + f"/weights-FC{celeb_df.num_features}-MobileNetV2" + "{val_binary_accuracy:.2f}.hdf5"
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True)

In [15]:
history = cls.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    max_queue_size=1,
    callbacks = [earlystop , model_checkpoint],
    verbose=1)

Epoch 1/5
2533/2533 [==============================] - 2631s 1s/step - loss: 0.3140 - binary_accuracy: 0.8628 - val_loss: 0.8768 - val_binary_accuracy: 0.8030
Epoch 2/5
2533/2533 [==============================] - 2564s 1s/step - loss: 0.2432 - binary_accuracy: 0.8930 - val_loss: 0.3869 - val_binary_accuracy: 0.8499
Epoch 3/5
2533/2533 [==============================] - 2358s 931ms/step - loss: 0.2279 - binary_accuracy: 0.8996 - val_loss: 0.8379 - val_binary_accuracy: 0.7580
Epoch 4/5
2533/2533 [==============================] - 2366s 934ms/step - loss: 0.2195 - binary_accuracy: 0.9032 - val_loss: 0.5494 - val_binary_accuracy: 0.8176
Epoch 5/5
2533/2533 [==============================] - 2361s 932ms/step - loss: 0.2143 - binary_accuracy: 0.9055 - val_loss: 0.4308 - val_binary_accuracy: 0.8532


In [16]:
os.mkdir("/kaggle/working/models")



path = "/kaggle/working/models/My_celebA_attr_Classifier_model"
cls.save(path)

In [22]:
os.mkdir("/kaggle/working/models/weights")
cls.save_weights('/kaggle/working/models/weights')